In [ ]:
import ast
import pandas as pd
from dotenv import load_dotenv
import os
from api import get_token, get_camera_token
from pyroclient import Client
import glob
from PIL import Image
import numpy as np
import io
import itertools
import requests
import random
import shutil
from utils import dl_seqs_from_url, read_pred_file, send_triangulated_alerts
import time

In [ ]:
API_URL = "http://api:5050"
load_dotenv("../.env")
SUPERADMIN_LOGIN = os.environ.get("SUPERADMIN_LOGIN")
SUPERADMIN_PWD = os.environ.get("SUPERADMIN_PWD")

# Get access token
admin_access_token = get_token(API_URL, SUPERADMIN_LOGIN, SUPERADMIN_PWD)


In [ ]:
users = pd.read_csv("../data/csv/API_DATA_DEV - users.csv")
cameras = pd.read_csv("../data/csv/API_DATA_DEV - cameras.csv")
users

In [ ]:
cameras

## Send some random alerts sequences from a list of cameras
Download a file containing relevant images and predictions, then send alerts to selected cameras (with a random azimuth and random imgs picked from the alerts folder downloaded below).

In [ ]:
send_alert_from_cam_ids = [2, 4, 5, 14] # select cameras

In [ ]:
SAMPLE_PATH = "selection-true-positives"
url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/selection-true-positives.zip"

if not os.path.isdir(SAMPLE_PATH):
    print("Images not found, dowloading ...")
    
    os.makedirs(SAMPLE_PATH, exist_ok=True)
    dl_seqs_from_url(url, SAMPLE_PATH)
else:
    print(f"SAMPLE_PATH {SAMPLE_PATH} directory exists, assume alerts inside, no download")

In [ ]:
sequances_folders = glob.glob(f"{SAMPLE_PATH}/*")

for camera_id in send_alert_from_cam_ids:
    
    camera_token = get_camera_token(API_URL, camera_id, admin_access_token)
    camera_client  = Client(camera_token, API_URL)

    sequances_folder = sequances_folders[random.randint(0,len(sequances_folders)-1)]
    imgs = glob.glob(f"{sequances_folder}/images/*")
    imgs.sort()
    preds = glob.glob(f"{sequances_folder}/labels_predictions/*")
    preds.sort()
    
    cam_center_azimuth = random.randint(0,360)
    print(f"Sending alerts from camera {camera_id} at azimuth {cam_center_azimuth}")
    for img_file, pred_file in zip(imgs, preds):
    
        stream = io.BytesIO()
        im = Image.open(img_file)
        im.save(stream, format="JPEG", quality=80)

        bboxes = read_pred_file(pred_file)
        response = camera_client.create_detection(stream.getvalue(), cam_center_azimuth, bboxes)
        # Force a KeyError if the request failed
        
        response.json()["id"]

## Download alerts sequences


In [ ]:
BASE_DIRECTORY = "../data"

SAMPLE_PATH = "alert_samples"
url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/alert_samples.zip"

if not os.path.isdir(os.path.join(BASE_DIRECTORY, SAMPLE_PATH)):
    print("Images not found, dowloading ...")
    #os.makedirs(SAMPLE_PATH, exist_ok=True)
    dl_seqs_from_url(url, os.path.join(BASE_DIRECTORY, SAMPLE_PATH))
else:
    print(f"SAMPLE_PATH {SAMPLE_PATH} directory exists, assume alerts inside, no download")



## Send custom alerts sequences

The  following code block helps you send some alerts/sequences from a specific diretory containing sequences data

- Create a directory with the name you want, ex ```alerts_with_style```
- In directory ```../data```or in current dir, there are some alerts folders downloaded
- Copy each inteterting alert dir (dir that contains a dir img, and a dir label predictions) and paste them to your ```alerts_with_style``` directory
- then below, fill the ```SEQUENCES_DIR_PATH``` with the path to you directory
- fill the CAM_MAPPING mapping dict : the first number in alert folder are dict keys that will map with camera id in database :  ex {2:78} -> all alert folder starting with 2 will create alerts for the camera of id 78
     - you can find cameras id in the object ```cameras``` (created at the beginning of this notebook), or look in the DB table cameras
     - If you put alert dir with prefix number that does not match, no alerts will be created
  

In [ ]:
SEQUENCES_DIR_PATH = "../data/alert_samples/single_sequences/*"
sequences_directories = glob.glob(SEQUENCES_DIR_PATH)
CAM_MAPPING = {22:13, 42:7, 59:5, 14:11, 43:8, 79:14, 13:10, 11:15, 59:1, 60:2, 15:12, 12:9, 10:16, 41:2, 65:8}

for seq_path in sequences_directories:
    
    seq_prefix_num = int(seq_path.split("/")[-1].split("_")[0])

    if CAM_MAPPING.get(seq_prefix_num) is None:
        print(f" === WARNING the prefix {seq_prefix_num} is not in CAM_MAPPING dict, hence no alerts will be generated (or from random camera but no implemented yet)")
    else:    
        camera_id = CAM_MAPPING[seq_prefix_num]
        camera_token = get_camera_token(API_URL, camera_id, admin_access_token)
        camera_client  = Client(camera_token, API_URL)
    
        imgs = glob.glob(f"{seq_path}/images/*")
        imgs.sort()
        preds = glob.glob(f"{seq_path}/labels_predictions/*")
        preds.sort()

        cam_center_azimuth = random.randint(0,360)
        
        print(f"Sending alerts from camera {camera_id} at azimuth {cam_center_azimuth}")
        for img_file, pred_file in zip(imgs, preds):
        
            stream = io.BytesIO()
            im = Image.open(img_file)
            im.save(stream, format="JPEG", quality=80)
    
            #bboxes = read_pred_file(pred_file)
            with open(pred_file, 'r', encoding='utf-8') as file:
                bboxes =  ast.literal_eval(file.read())
    
            #bboxes = np.loadtxt(pred_file, ndmin=2)
            response = camera_client.create_detection(stream.getvalue(), cam_center_azimuth, bboxes)
            # Force a KeyError if the request failed
            #time.sleep(1)
            response.json()["id"]

## Send 2 sequences that triangulate
Download a file containning 2 alerts from 2 cameras that triangulate, then it send alerts to the api. 

In [ ]:
if not os.path.isdir("triangulated_sequences"):
    print("Images not found, dowloading ...")
    output_path = "triangulated_sequences.zip"

    url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/"+"triangulated_sequences.zip"

    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raises an error for bad status codes

    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    zip_path = output_path
    extract_dir = "triangulated_sequences"  # Current directory

    shutil.unpack_archive(zip_path, extract_dir, 'zip')

    print("Extraction completed.")

cam_triangulation = {
    "12": {
        "azimuth": 226,
        "path": "triangulated_sequences/brison-03"
    },
    "13": {
        "azimuth": 54,
        "path": "triangulated_sequences/serre-de-barre-01"
    }
}

send_triangulated_alerts(cam_triangulation, API_URL, Client, admin_access_token)


## Send 3 sequences that triangulates, then 2 other sequences that triangulates

In [ ]:
cam_triangulation = {
    "4": {
        "azimuth": 163.3878,
        "path": "../data/alert_samples/triangulated_sequences/41_croix-augas-02_2025-07-14_12-25-03"
    },
    "8": {
        "azimuth": 8.2793,
        "path": "../data/alert_samples/triangulated_sequences/65_nemours-02_2025-07-14_11-57-39"
    },
    "5": {
        "azimuth": 276.5041,
        "path": "../data/alert_samples/triangulated_sequences/66_moret-sur-loing-01_2025-07-14_12-33-39"
    }
}


send_triangulated_alerts(cam_triangulation, API_URL, Client, admin_access_token)

cam_triangulation = {
    "11": {
        "azimuth": 163.3878,
        "path": "../data/alert_samples/triangulated_sequences/14_brison-03_2025-09-29_14-02-22"
    },
    "13": {
        "azimuth": 8.2793,
        "path": "../data/alert_samples/triangulated_sequences/22_serre-de-barre-01_2025-09-29_14-23-28"
    }
}


send_triangulated_alerts(cam_triangulation, API_URL, Client, admin_access_token)